In [ ]:
from tqdm import tqdm
import requests
import pandas as pd
import numpy as np

In [ ]:
assays = [
    "ATG_PPARg_TRANS_dn", 
    "ATG_PPARg_TRANS_up", 
    "TOX21_PPARg_BLA_antagonist_ratio", 
    "TOX21_PPARg_BLA_Agonist_ratio", 
    "NVS_NR_hPPARg", 
    "ERF_ENZ_hCYP19A1_dn", 
    "NVS_ADME_hCYP19A1", 
    "NVS_ADME_hCYP19A1_Activator", 
    "TOX21_Aromatase_Inhibition", 
    "CEETOX_H295R_ESTRADIOL_dn", 
    "CEETOX_H295R_ESTRADIOL_noMTC_dn", 
    "CEETOX_H295R_ESTRADIOL_noMTC_up", 
    "CEETOX_H295R_ESTRADIOL_up", 
    "CEETOX_H295R_ESTRONE_dn", 
    "CEETOX_H295R_ESTRONE_noMTC_dn", 
    "CEETOX_H295R_ESTRONE_noMTC_up", 
    "CEETOX_H295R_ESTRONE_up"
]

base_url = "https://comptox.epa.gov/dashboard-api/ccdapp2/assay-list-details/search/by-endpointname?name=[name]&size=[size]&page=[page]"

In [ ]:
for assay in assays:
    jsons = []
    probe_resp = requests.get(base_url
                              .replace("[name]", assay.strip())
                              .replace("[page]", "1")
                              .replace("[size]", "1"))
    elements = int(probe_resp.json()['page']['totalElements'])
    pages = elements // 1000 + 1
    print(f"{assay} -> Total elements: {elements}, pages: {pages}")
    for page in range(0, pages):
        resp = requests.get(base_url
                                  .replace("[name]", assay.strip())
                                  .replace("[page]", str(page))
                                  .replace("[size]", "1000"))
        if resp.status_code == 200:
            print(f"{assay} -> HTTP{resp.status_code}, Page: {page}")
            jsons.append(resp.json())
        else:
            print(f"ERROR: {assay} -> HTTP{resp.status_code}")

    json = jsons[0]['_embedded']['assayListDetails']

    for i in range(1, len(jsons)):
        json.extend(jsons[i]['_embedded']['assayListDetails'])

    df = pd.DataFrame(json)

    assert len(df) == elements

    path = f"data/{assay}.csv"
    print(f"Saved {assay} -> {path}")
    df.to_csv(path, index=False)
